# 5. Evolve with SUPG, timedep

In [2]:
notes = "\n SUPG: BDF2 using P2 and halving timestep at each refinement level \n\n"

from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

# ------------------------------------------ #

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

def evolve_t(folder, nx, dt, T, degree, u_code, f_code, sigma, mu, velocity):
    t = 0.0
    u_exact = Expression(u_code, degree = degree+1, t = t)
    f = Expression(f_code, degree = degree+1, t = t)

#     mesh = UnitSquareMesh(nx,nx)
    mesh = RectangleMesh(Point(-2.0,-2.0), Point(2.0,2.0), 2*nx, 2*nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression('0', degree = degree+1, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n0 = interpolate(u_D, Q)
    u_n1 = Function(Q)
    u_ = Function(Q)

        # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)
    
# -------need u_n1------------------------------- #

    # Galerkin variational problem
    F = (u - u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)
    
    # SUPG stabilization terms
    h = CellDiameter(mesh)

    # based on paper's definition of residual and stabilization term
    Lt = -mu*div(grad(u)) + dot(velocity, grad(u)) + (sigma+1.0/dt)*u 
    ft = u_n0/dt + f
    r = ft - Lt
    vnorm = sqrt(dot(velocity, velocity))
    F -= dt*(h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx    
    
    
    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    t += dt
    
    u_.rename('u','u')
    
    u_D.t = t
    f.t = t
    u_exact.t = t
    velocity.t = t

    b = assemble(L)
    bc = DirichletBC(Q, u_D, boundary)
    bc.apply(A1)
    bc.apply(b)
    
    solve(A1, u_n1.vector(), b)#, 'gmres')
    progress.update(t / T)

    
    # -------------------------------------- #

    # Galerkin variational problem
    F = (1.5*u - 2.0*u_n1 + 0.5*u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)
    
    # based on paper's definition of residual and stabilization term
    Lt = -mu*div(grad(u)) + dot(velocity, grad(u)) + (sigma+1.5/dt)*u 
    ft = 2.0*u_n1/dt - 0.5*u_n0/dt + f
    r = ft - Lt
    vnorm = sqrt(dot(velocity, velocity))
    F -= dt*(h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx    

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)


    
    # Outputting files
    out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    out_file_ubar = File(folder+"SUPG_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    
    u_.rename('u','u')
    
    # Save t = 0.0
#     out_file_uexact << (ue, float(t))
#     out_file_ubar << (u_, float(t))
    it = 0
    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt
        
        u_.rename('u','u')
        u_exact.rename('u','u')
        
        u_D.t = t
        f.t = t
        u_exact.t = t
        velocity.t = t
        # Assemble matrices
        
        A1 = assemble(a1)
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b)#, 'gmres')
        progress.update(t / T)
        
                
        if it % 5:
            out_file_ubar << (u_, float(t)) 
                
        it += 1
        
        u_n0.assign(u_n1)
        u_n1.assign(u_)

    L2, H1 = compute_errors(u_exact, u_, t, mesh)
    maxval, minval = compute_extrema(u_, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
    return str(L2)+','+str(H1)+','+str(maxval)+','+str(minval)

# ------------------------------------------ #

from datetime import datetime, date

def run(nx,dt,sigma,mu,folder):
    d_date = datetime.now()
    today = d_date.strftime('%Y-%m-%d')
    f = open(today+'_log_SUPG.txt',"a+")
    
    T = 3
    degree = 2

    velocity = Expression(('cos(t)','sin(t)'), degree = degree, t = 0)
    adr_f = Expression('exp(-(pow(x[0],2)+pow(x[1],2))/pow(0.07,2))', degree = degree)


    sim_data = 'SUPG, dt='+str(dt)+', T='+str(T)+', P'+str(degree)
    f.write(sim_data+'\n')


    now = d_date.strftime("%I:%M:%S %p")
    f.write('start: '+now+'\n\n')

    output = evolve_t(folder, nx, dt, T, degree, '0', adr_f.cppcode, sigma, mu, velocity)
    f.write(str(nx)+','+str(dt)+','+output+'\n')


    d_date = datetime.now()
    now = d_date.strftime("%I:%M:%S %p")
    f.write('\nend: '+now+'\n\n')
    f.write(notes)

    f.close()

In [3]:
dt=0.01
sigma = 0.1
mu = 0.00005
folder = 'dt1e-2_2x2_Dir0/SUPG/sigma'+str(sigma)+'_mu'+str(mu)
run(25,dt,sigma,mu,folder)
run(50,dt,sigma,mu,folder)
run(100,dt,sigma,mu,folder)
run(200,dt,sigma,mu,folder)
run(400,dt,sigma,mu)

25 , 0.0509206554446 , 0.666403936179 , 0.112361765766 , -0.0108051822084
50 , 0.0527382005666 , 0.777866784326 , 0.122334353915 , -0.0123998392895
100 , 0.0529467007902 , 0.847877153749 , 0.121674123958 , -0.0133267718164
200 , 0.052967656972 , 0.943356562505 , 0.121783066498 , -0.0134010080821


TypeError: run() takes exactly 5 arguments (4 given)